In [12]:
import os
from pathlib import Path
import pandas as pd

In [14]:
# To do later: Fetching data file automatically
DirPpath = Path(os.path.abspath('')).parent # Fetching the current directory path - Specific for ipynb file - For .py: Path(os.path.dirname(os.path.realpath(__file__)).replace("\\", "/"))

PledgesCsvPath = str(DirPpath.absolute()) + "/DataExploration/CleanedData.csv"  

In [17]:
PledgesDf = pd.read_csv(PledgesCsvPath, index_col=0)

In [58]:
import string
import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet

In [34]:
def first_clean(text):
    return " ".join(text.split()).replace("_x000D_","")

first_clean(PledgesDf.iloc[3,1]) # Delete the returns from excel

'Achieve sustainable and flexible solutions for multimodal transport and develop policies to protect natural heritage and biodiversity, respecting the socio-cultural authenticity of host communities.  CNA Turismo e Commercio, over the three-year period (between autumn 2022 and throughout 2023), will organize training seminars for businesses - with the involvement of public and private stakeholders - aimed at the implementation of concrete solutions for the development of good practices for a supply of multimodal transport and protocols for the respect and protection of the natural heritage and biodiversity.'

In [52]:
def preprocess(text):
    text = text.lower() # Lowercase all the characters from the string
    text = text.strip() # Remove the leading and trailing whitespaces
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text) # Removing Punctuation
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text)) # Remove non alphanumeric characters
    text = re.sub(r'\d', '', text) # Removing digits
    text = re.sub(r'\s+', ' ', str(text).strip()) # Replacing "double, triple, etc" whitespaces by one
    return text

preprocess(first_clean(PledgesDf.iloc[3,1]))

'achieve sustainable and flexible solutions for multimodal transport and develop policies to protect natural heritage and biodiversity respecting the socio cultural authenticity of host communities cna turismo e commercio over the three year period between autumn and throughout will organize training seminars for businesses with the involvement of public and private stakeholders aimed at the implementation of concrete solutions for the development of good practices for a supply of multimodal transport and protocols for the respect and protection of the natural heritage and biodiversity'

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [68]:
def stopword(string):
    a = [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

stopword(preprocess(first_clean(PledgesDf.iloc[3,1])))

'achieve sustainable flexible solutions multimodal transport develop policies protect natural heritage biodiversity respecting socio cultural authenticity host communities cna turismo e commercio three year period autumn throughout organize training seminars businesses involvement public private stakeholders aimed implementation concrete solutions development good practices supply multimodal transport protocols respect protection natural heritage biodiversity'

In [71]:
# LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()


# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(nltk.word_tokenize(string))  # Get position tags
    a = [wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in
         enumerate(word_pos_tags)]  # Map the position tag and lemmatize the word/token
    return " ".join(a)

lemmatizer(stopword(preprocess(first_clean(PledgesDf.iloc[3,1]))))

'achieve sustainable flexible solution multimodal transport develop policy protect natural heritage biodiversity respect socio cultural authenticity host community cna turismo e commercio three year period autumn throughout organize training seminar business involvement public private stakeholder aim implementation concrete solution development good practice supply multimodal transport protocol respect protection natural heritage biodiversity'

In [77]:
def preprocessing(text):

    global n
    n = n +1
    
    print("**************")
    print("n is : ")
    print(n)
    print("length of the text is : ")
    print(len(first_clean(text)))
    return lemmatizer(stopword(preprocess(first_clean(text))))

In [ ]:
n=0
print("begin pre-processing")
PledgesDf['clean_text'] = PledgesDf['Pledge'].apply(lambda x: preprocessing(x))
print("end pre-processing")

In [80]:
PledgesDf.head()

,Topic,Pledge,clean_text
0,1,HOTREC calls for a level playing field and fai...,hotrec call level play field fair competition ...
1,1,Actually we as an association are still pretty...,actually association still pretty much begin d...
2,1,Estonia sees the need to synchronize and harmo...,estonia see need synchronize harmonize rule sh...
3,2,Achieve sustainable and flexible solutions for...,achieve sustainable flexible solution multimod...
4,2,The Ile-de-France Region aims to make tourist ...,ile de france region aim make tourist site acc...


In [122]:
tokens = [nltk.word_tokenize(i) for i in PledgesDf["clean_text"]]

In [98]:
word_freq = {}
for sent in tokens:
    for i in sent:

        if i not in word_freq.keys():
            word_freq[i] = 1
        else:
            word_freq[i] += 1
len(word_freq)

3853

In [99]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['tourism',
 'sustainable',
 'destination',
 'de',
 'development',
 'data',
 'project',
 'support',
 'sustainability',
 'sector']

In [84]:
import gensim
from gensim.models import Word2Vec

In [123]:
# Setting the model with its hyperparameters
model = Word2Vec(sentences = tokens, vector_size=100, window = 5)

In [124]:
model.wv.get_vector("hotrec")

array([-0.03200323,  0.04947015,  0.0303732 ,  0.01229963, -0.00495085,
       -0.08394027,  0.02444905,  0.09911132, -0.02422472, -0.00613349,
       -0.02879979, -0.05694728, -0.00317937,  0.0193244 , -0.01179363,
       -0.05869448, -0.00354247, -0.08079367, -0.03086811, -0.06268363,
        0.03832952,  0.03726424,  0.04147314, -0.02776401, -0.01850415,
       -0.00827994, -0.04463762, -0.03819501, -0.01409421,  0.01621452,
        0.05203019,  0.00444728,  0.01470546, -0.001519  , -0.02608609,
        0.05959416,  0.00102265, -0.0491998 , -0.01705319, -0.08718075,
        0.01625505, -0.02980288, -0.0290526 , -0.00953743,  0.05582019,
       -0.01028011, -0.03645533, -0.01349658,  0.02922405,  0.00950451,
        0.00265618, -0.04982678,  0.03399677, -0.01487433, -0.04223935,
        0.04377802,  0.02429913, -0.0070756 , -0.03900617,  0.01343666,
        0.02140767,  0.01819231, -0.01605036, -0.05664711, -0.04864142,
        0.03207529,  0.02343914,  0.05991454, -0.05697872,  0.02

In [125]:
model.wv.most_similar(positive = ["sustainable"])

[('tourism', 0.9997771382331848),
 ('sustainability', 0.9997443556785583),
 ('measure', 0.9997432231903076),
 ('plan', 0.9997410774230957),
 ('strategy', 0.9997352957725525),
 ('eu', 0.9997327327728271),
 ('travel', 0.9997319579124451),
 ('service', 0.9997313618659973),
 ('target', 0.9997299909591675),
 ('develop', 0.999729573726654)]

In [127]:
import gensim.downloader as api

In [128]:
wv2 = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [130]:
wv2.most_similar(positive=["hotrec"])

KeyError: "Key 'hotrec' not present in vocabulary"